In [13]:
import boto3
import networkx as nx
import numpy as np
import uuid
import time

import os
import io
import configparser
import requests

In [14]:
config = configparser.ConfigParser()

# Write example config to disk.
config['AWS'] = {
    'ACCESS_KEY': '',
    'SECRET_KEY': '',
    'SESSION_TOKEN': ''
}

config['Master'] = {
    'MASTER_URL': '',
    'API_KEY': ''
}

with open('config.txt.sample', 'w') as configfile:
    config.write(configfile)
    
config = configparser.ConfigParser()
config.read('config.txt')

session = boto3.Session(
    aws_access_key_id=config['AWS']['access_key'], 
    aws_secret_access_key=config['AWS']['secret_key'], 
)


In [18]:
def getInstanceIdsByType():
    response = session.client("ec2").describe_instances()
    reservations = response['Reservations']
    
    instancesByType = dict()
    for reservation in reservations:
        for instance in reservation['Instances']:
            for tag in instance['Tags']:
                if tag['Key'] == 'instance_type':
                    instancetype = tag['Value']
                    if not instancetype in instancesByType:
                        instancesByType[instancetype] = [instance['InstanceId']]
                    else:
                        instancesByType[instancetype].append(instance['InstanceId'])
    
    return instancesByType    

def terminateInstancesWithIds(ids):
    session.client("ec2").terminate_instances(InstanceIds=ids)

def createMaster(session):
    ec2_client = session.client('ec2')

def generate_graph_file(directory, n, p, prepend_vertex_weights=False):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    file = os.path.join(directory, '%s-%d-%s.graph' % (uuid.uuid4(), n, p))

    # Generate ER graph with n nodes and edge probability p
    G = nx.erdos_renyi_graph(int(n), p)
    node_weights = []

    # Generate random edge weights.
    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = round(np.random.random(), 2)

    nx.write_weighted_edgelist(G, file, delimiter=",")

    if prepend_vertex_weights:
        # Prepend weights
        for u in G.nodes():
            node_weights.append(str(1.0 / n)) # Uniform for pagerank.

        with open(file, 'r') as original: data = original.read()
        with open(file, 'w') as modified: modified.write(",".join(node_weights) + "\n" + data)
            
    return file

def generate_graph(output, n, p, prepend_vertex_weight=False):    
    if prepend_vertex_weight:
        # Uniform for pagerank.
        node_weights = [str(1.0 / n)] * n
        output.write(output.write(",".join(node_weights) + "\n"))

    # Generate ER graph with n nodes and edge probability p
    G = nx.erdos_renyi_graph(int(n), p)
    
    # Generate random edge weights.
    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = round(np.random.random(), 2)
        
    node_weights = []

    nx.write_weighted_edgelist(G, output, delimiter=",")

def run_experiment():
    api_url = config['Master']['master_url']
    api_key = config['Master']['api_key']
    max_workers = 2
    
    master_user_data = """
    #!/bin/bash

    curl -L https://github.com/doriandekoning/IN4392-cloud-computing-lab/releases/download/$(curl --silent "https://api.github.com/repos/doriandekoning/IN4392-cloud-computing-lab/releases/latest" | grep -Po '"tag_name": "\K.*?(?=")')/master_amd64_linux  > /home/ubuntu/go/bin/master && chmod +x /home/ubuntu/go/bin/master

    echo 'export MAXWORKERS=%s' > /home/ubuntu/env
    echo 'export MASTER_PORT=8000' >> /home/ubuntu/env
    echo 'export MASTER_ADDRESS=%s' >>/home/ubuntu/env
    echo 'export OWN_PORT=8888' >> /home/ubuntu/env
    echo 'export OWN_ADDRESS=http://`wget http://ipecho.net/plain -O - -q ; echo`' >> /home/ubuntu/env
    echo 'export API_KEY=%s' >> /home/ubuntu/env
    echo 'export OWN_INSTANCEID=`ec2metadata --instance-id`' >> /home/ubuntu/env
    """ % (max_workers, api_url, api_key)
    
    print(master_user_data)
    return
    
    try:
        # Connect to EC2
        ec2_client = session.client('ec2')

        # Spin up a master.
        response = ec2_client.run_instances(LaunchTemplate={'LaunchTemplateName': "Master"}, MinCount=1, MaxCount=1, UserData=master_user_data)
        
        initialized = False
        # Wait until the master is ready.
        while not initialized:
            # Attempt to contact /health, if we manage to we are ready.
            try:
                r = requests.get('%s/health' % api_url, headers={'X-Auth': api_key}, timeout=5)
                
                if r.status_code is 200:
                    initialized=True
                    print("Master is ready!")

            except: 
                print("Waiting for master...")
                continue

        # Generate graphs and post them to the system.
        for i in range(10):
            output = io.BytesIO()
            generate_graph(output, 10, 0.5)
            graph = output.getvalue().decode('utf-8')

            try:
                r = requests.post('%s/processgraph?algorithm=pagerank&maxsteps=300' % api_url, headers={'X-Auth': api_key}, timeout=20, data=graph)
                r.raise_for_status()
            except:
                print("Error submitting graph for processing.")
            

        # If all graphs are posted we start polling every 10 seconds if the system is done.
        finished = False
        while not finished:
            # Every 10 seconds, check if we're done.
            time.sleep(10)
            finished = True
            print("Master has finished processing...")

            
        # Make sure all workers are being killed.
        r = requests.get('%s/killworkers' % api_url, headers={'X-Auth': api_key}, timeout=20)
        # Make sure all logs are being written.
        # TODO.
        

    finally:
        # Sleep so the master can finish/kill workers.
        time.sleep(10)
        
        print("Terminating master...")
        # Terminate all masters.
        instances = getInstanceIdsByType()
        terminateInstancesWithIds(instances['master'])
        
        # Also terminate workers to be sure we clean everything up.
        terminateInstancesWithIds(instances['worker'])

        print("Finished...")


In [19]:
run_experiment()

# # Terminate all masters.
# instances = getInstanceIdsByType()
# terminateInstancesWithIds(instances['master'])


    #!/bin/bash

    curl -L https://github.com/doriandekoning/IN4392-cloud-computing-lab/releases/download/$(curl --silent "https://api.github.com/repos/doriandekoning/IN4392-cloud-computing-lab/releases/latest" | grep -Po '"tag_name": "\K.*?(?=")')/master_amd64_linux  > /home/ubuntu/go/bin/master && chmod +x /home/ubuntu/go/bin/master

    echo 'export MAXWORKERS=2' > /home/ubuntu/env
    echo 'export MASTER_PORT=8000' >> /home/ubuntu/env
    echo 'export MASTER_ADDRESS=http://in4392.ddns.net:8000' >>/home/ubuntu/env
    echo 'export OWN_PORT=8888' >> /home/ubuntu/env
    echo 'export OWN_ADDRESS=http://`wget http://ipecho.net/plain -O - -q ; echo`' >> /home/ubuntu/env
    echo 'export API_KEY=Hv5jxHt1x2E6Z1Ofyo1gBcQKopUTOd48v2w4APmHpw6RR10EvN' >> /home/ubuntu/env
    echo 'export OWN_INSTANCEID=`ec2metadata --instance-id`' >> /home/ubuntu/env
    


In [49]:
# directory = 'graphs'
# node = [1e1, 1e2, 1e3]
# p_edge = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# # For each node_vertex combination
# for n in node:
#     for p in p_edge:
#         generate_graph_file(directory, n, p, prepend_vertex_weights=True)